In [ ]:
import pandas as pd

def load_data(filename='train.csv'):
    df = pd.read_csv(filename)
    return df

train = load_data()
test = load_data('test.csv')

train['train_flag'] = 1
test['train_flag'] = 0

data = pd.concat([train, test], ignore_index=True)  #data is the combined one

# Columns to drop
✅ PassengerId - No predictive value, just ID
✅ Name - Unique strings, drop after extracting Title  
✅ Ticket - Mostly unique, low value
✅ Cabin - 77% missing, drop (or create HasCabin later)

# columns to keep
❌ DON'T drop Pclass - **Strong predictor** (3rd class = lower survival)
❌ DON'T drop Age - **Fill missing** (median by Pclass/Sex), then bin into groups
❌ DON'T drop Fare - **Keep** (higher fare = higher survival chance) 
✅ Survived - Target (train only)
✅ Sex - **Best predictor** (women > men survival)
✅ SibSp - Siblings/spouses on board
✅ Parch - Parents/children on board  
✅ Embarked - Port of origin (some signal)
